In [1]:
#Cargamos Diccionario y Vector de Corpus
#Cargar Matriz de Documentos
from gensim import corpora

class Documentos(object):
    def __iter__(self):
        for line in open('../Data/resumenes.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())
dictionary = corpora.Dictionary().load('../Data/resumenes.dict')
corpus = corpora.MmCorpus('../Data/resumenes_vectores.mm')

## Usando TFIDF 

In [2]:
from gensim import models

#model = models.TfidfModel(corpus, normalize=True)
tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model (train)
corpus_tfidf = tfidf[corpus]
corpus_tfidf # step 2 -- use model and transform 

In [3]:
#LSI Model
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [5]:
lsi_model.print_topic(200)

'0.151*"midy" + -0.130*"carotenos" + 0.093*"mercurio" + 0.071*"fronteras" + -0.071*"nps" + -0.070*"npk" + -0.064*"fiesta" + 0.064*"perspicillata" + 0.062*"malezas" + -0.059*"servidor"'

In [33]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
i = 0
#similitud de documentos con algunos temas
for doc, as_text in zip(corpus_lsi, documents):
    if i > 10: break
    i = i + 1
    print(doc)

[(0, -0.15382514037268388), (1, -0.07854030654456619)]
[(0, -0.1514677298505903), (1, -0.00515288272186968)]
[(0, -0.12971521476467093), (1, 0.07311514039117664)]
[(0, -0.1287706046702057), (1, 0.08933811192731118)]
[(0, -0.07099970355538221), (1, 0.0121671475286895)]
[(0, -0.15838692218485023), (1, 0.10788923865545308)]
[(0, -0.1845097489944218), (1, -0.012200192053673025)]
[(0, -0.1894718453855591), (1, -0.01711265753192467)]
[(0, -0.2362566720921067), (1, -0.006394699529057206)]
[(0, -0.1384802727006627), (1, 0.06023777858949483)]
[(0, -0.15811591138959735), (1, -0.03818489577116049)]


In [57]:
doc100 = corpus_lsi[424]
for topic in doc100:
    if topic[1] > 0.1:
        print(topic)

(28, 0.16371342176435547)
(39, 0.1547042196019552)
(45, 0.16588130763224893)
(49, 0.11926113229711457)
(50, 0.11672096451571883)
(260, 0.10231052851834882)


## Queries

In [4]:
doc = "saber pro"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi_model[vec_bow]  # convert the query to LSI space
print(len(vec_lsi))

300


In [5]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi_model[corpus])  # transform corpus to LSI space and index it

In [20]:
index.save('/tmp/resumenes.index')
index = similarities.MatrixSimilarity.load('/tmp/resumenes.index')

In [6]:
# resumenes = Documentos()

sims = index[vec_lsi]  # perform a similarity query against the corpus
results = sorted(list(enumerate(sims)), key=lambda item: -item[1])
for i, documento in enumerate(results):
    if(documento[1] > 0.1):
        print(documento)
# print(match[421])  # print (document_number, document_similarity) 2-tuples


(14, 0.6551641)
(13, 0.6400207)
(20, 0.449788)
(219, 0.32026112)
(205, 0.18345365)
(23, 0.17499557)
(250, 0.1744338)
(105, 0.16220774)
(148, 0.15405932)
(64, 0.1500764)
(310, 0.14978106)
(116, 0.13722216)
(162, 0.11209622)
(406, 0.106378675)
(71, 0.10414532)
